In [2]:
import os.path
%run D:\workspaces\General\notebooks\image-processing\0-0-common.ipynb

In [3]:
%%time

subs = ["SFWRedheads","sfwpetite","SFWNextDoorGirls", "celebrities", "wallstreetbets", "fatsquirrelhate","CityPorn","EarthPorn","memes","spaceporn","trippinthroughtime","fatsquirrelhate","oldladiesbakingpies","itookapicture", "celebrities", "realasians", "KoreanHotties", "prettyasiangirls","AsianOfficeLady", "mildlypenis"]

extant_data: pd.DataFrame = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

distinct_data: pd.DataFrame = extant_data.drop_duplicates(subset=['id', 'title']).reset_index(drop=True)

target_frame: pd.DataFrame = distinct_data[distinct_data['subreddit'].isin(subs)]

sorted_target_frame = target_frame.sort_values(by=['title'], ascending=False)

display(sorted_target_frame)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
2755,10yvvbe,trippinthroughtime,Reddit_Survivor9,🫡,a painting of a man in a black dress and a wom...,4c6389236f78e168e69a4dbef104ddbc,/r/trippinthroughtime/comments/10yvvbe/_/,https://i.redd.it/mixpfihld9ha1.jpg,mixpfihld9ha1.jpg,/data/images/trippinthroughtime/mixpfihld9ha1.jpg,/data/images/trippinthroughtime/thumbnail/mixp...,True,False
17774,10d63d9,EarthPorn,Big-Vermicelli-6292,🧚‍♀️Fortnite For the Night🕹🤍,a woman with a cat ears and ears is wearing a ...,a5a81096971dba9efdead241f17b0110,/r/EarthPorn/comments/10d63d9/fortnite_for_the...,https://i.redd.it/yh8bfpmrtdca1.jpg,a5a81096971dba9efdead241f17b0110.jpg,D:\data\images\EarthPorn\a5a81096971dba9efdead...,D:\data\images\EarthPorn\thumbnail\a5a81096971...,True,False
18264,znyfkm,memes,Youngsanchoz,🥶🥶🥶,a penguin with a basketball ball in his hand a...,90e75078331dd3f529cb1f7fd4ec1754,/r/memes/comments/znyfkm/_/,https://i.redd.it/1gbkg72ekf6a1.jpg,90e75078331dd3f529cb1f7fd4ec1754.jpg,D:\data\images\memes\90e75078331dd3f529cb1f7fd...,D:\data\images\memes\thumbnail\90e75078331dd3f...,True,False
27501,11p6qw3,KoreanHotties,tommybour,🥵🥵,None,None,/r/KoreanHotties/comments/11p6qw3/_/,https://i.redd.it/rfwsl0x8gana1.png,.jpg,None,None,False,False
27978,11h5qq7,KoreanHotties,Double_Ad9489,🥵🥵,None,None,/r/KoreanHotties/comments/11h5qq7/_/,https://www.reddit.com/gallery/11h5qq7,None,None,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29509,11qjwhd,mildlypenis,BoatsFA20,"""Boy Scout Trail"". Crescent City, CA",there is a bear that is sitting on a tree in t...,e82b29e86440b6d88e0a1bcfc6e6670a,/r/mildlypenis/comments/11qjwhd/boy_scout_trai...,https://i.redd.it/bw36vxpwslna1.jpg,e82b29e86440b6d88e0a1bcfc6e6670a.jpg,D:\data\images\mildlypenis\e82b29e86440b6d88e0...,D:\data\images\mildlypenis\thumbnail\e82b29e86...,True,False
9855,zoqvfz,spaceporn,Adeel314,"""Another day of sunspot AR12781. One of my fav...",a close up of a sunflower on a sunflowered sun...,6914cff6140c5c97db4acb778075cb0d,/r/spaceporn/comments/zoqvfz/another_day_of_su...,https://i.redd.it/etun5sh2dl6a1.jpg,etun5sh2dl6a1.jpg,/data/images/spaceporn/etun5sh2dl6a1.jpg,/data/images/spaceporn/thumbnail/etun5sh2dl6a1...,True,False
18645,yv6l1a,spaceporn,Strong-Ambassador792,"""A Galaxy on Edge"" - Knife Edge Galaxy","a view of the milky and the milky, with stars ...",46f16c15693b777dd825532b34cff69f,/r/spaceporn/comments/yv6l1a/a_galaxy_on_edge_...,https://i.redd.it/9guqcf72dyz91.jpg,46f16c15693b777dd825532b34cff69f.jpg,D:\data\images\spaceporn\46f16c15693b777dd8255...,D:\data\images\spaceporn\thumbnail\46f16c15693...,True,False
28825,1253gtx,mildlypenis,hihungryimdad_,"""A Flower"" (not op)",two trays with food items on them and a tray w...,f4e82b73d2c5e1ec7e0716997463e699,/r/mildlypenis/comments/1253gtx/a_flower_not_op/,https://i.redd.it/9i2knqladlqa1.jpg,f4e82b73d2c5e1ec7e0716997463e699.jpg,D:\data\images\mildlypenis\f4e82b73d2c5e1ec7e0...,D:\data\images\mildlypenis\thumbnail\f4e82b73d...,True,False


CPU times: total: 641 ms
Wall time: 1.15 s


In [4]:
%%time

target_frame['exists'] = target_frame.apply(lambda row: check_if_exists(row['thumbnail_path']), axis=1)

exists = target_frame.where(target_frame['exists']).dropna(how='all').reset_index(drop=True)

display(exists)

<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,128z5ns,celebrities,A-Sexy-Name,Rachel Weisz,cars are parked on the side of the road in the...,f09b1c0c5dea50c934081a04c83d4d33,/r/celebrities/comments/128z5ns/rachel_weisz/,https://i.redd.it/nv80afddacra1.jpg,f09b1c0c5dea50c934081a04c83d4d33.jpg,D:\data\images\celebrities\f09b1c0c5dea50c9340...,D:\data\images\celebrities\thumbnail\f09b1c0c5...,True,False
1,11yk8db,prettyasiangirls,MisoShiru520,Library,blonde woman with blonde hair and tattoos on h...,9d5b7331b941ecf3fdbc8ad322303825,/r/prettyasiangirls/comments/11yk8db/library/,https://i.redd.it/ht4d2wuusapa1.jpg,9d5b7331b941ecf3fdbc8ad322303825.jpg,D:\data\images\prettyasiangirls\9d5b7331b941ec...,D:\data\images\prettyasiangirls\thumbnail\9d5b...,True,False
2,11hfioi,KoreanHotties,Majestic_Painter8660,Underboob bikini,a man with a beard and a beard sitting in fron...,a154a56b1b87cf3a0bc93600c86e4585,/r/KoreanHotties/comments/11hfioi/underboob_bi...,https://i.redd.it/cimr17q84jla1.jpg,a154a56b1b87cf3a0bc93600c86e4585.jpg,D:\data\images\KoreanHotties\a154a56b1b87cf3a0...,D:\data\images\KoreanHotties\thumbnail\a154a56...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23823,11oz7fi,mildlypenis,Extra-Lie489,Lenis,there is a small piece of metal with two small...,7604c3e55ce8dbf238520683d6e71f89,/r/mildlypenis/comments/11oz7fi/lenis/,https://i.redd.it/pskj3q0ao8na1.jpg,7604c3e55ce8dbf238520683d6e71f89.jpg,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,D:\data\images\mildlypenis\thumbnail\7604c3e55...,True,False
23824,11oz73d,mildlypenis,DeimosLV,My friend is making a stuffed animal and this ...,a close up of a blue knitted bagueth on a chair,389f8aa0fb191bdeba08c8d2cfff8ce2,/r/mildlypenis/comments/11oz73d/my_friend_is_m...,https://i.redd.it/gemqhq67o8na1.jpg,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,D:\data\images\mildlypenis\thumbnail\389f8aa0f...,True,False
23825,11oyzm3,mildlypenis,Mymemesareswell,This weird Mickey keychain hand,someone is holding a key chain with a mickey m...,87563ac0c8ddeeb0ce47bc915b99308b,/r/mildlypenis/comments/11oyzm3/this_weird_mic...,https://i.redd.it/vrivt8rom8na1.jpg,87563ac0c8ddeeb0ce47bc915b99308b.jpg,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,D:\data\images\mildlypenis\thumbnail\87563ac0c...,True,False
23826,11oyvx1,mildlypenis,LucindaBobinda,Parking lot pp,a close up of a black and white photo of a bla...,88b596be6de959bd29eed4e8946bb476,/r/mildlypenis/comments/11oyvx1/parking_lot_pp/,https://i.redd.it/w1vebizwl8na1.jpg,88b596be6de959bd29eed4e8946bb476.jpg,D:\data\images\mildlypenis\88b596be6de959bd29e...,D:\data\images\mildlypenis\thumbnail\88b596be6...,True,False


CPU times: total: 4.84 s
Wall time: 7.53 s


In [5]:
exists.where(lambda x: x['caption'] != "").dropna(how='all').reset_index(drop=True)
display(exists)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,128z5ns,celebrities,A-Sexy-Name,Rachel Weisz,cars are parked on the side of the road in the...,f09b1c0c5dea50c934081a04c83d4d33,/r/celebrities/comments/128z5ns/rachel_weisz/,https://i.redd.it/nv80afddacra1.jpg,f09b1c0c5dea50c934081a04c83d4d33.jpg,D:\data\images\celebrities\f09b1c0c5dea50c9340...,D:\data\images\celebrities\thumbnail\f09b1c0c5...,True,False
1,11yk8db,prettyasiangirls,MisoShiru520,Library,blonde woman with blonde hair and tattoos on h...,9d5b7331b941ecf3fdbc8ad322303825,/r/prettyasiangirls/comments/11yk8db/library/,https://i.redd.it/ht4d2wuusapa1.jpg,9d5b7331b941ecf3fdbc8ad322303825.jpg,D:\data\images\prettyasiangirls\9d5b7331b941ec...,D:\data\images\prettyasiangirls\thumbnail\9d5b...,True,False
2,11hfioi,KoreanHotties,Majestic_Painter8660,Underboob bikini,a man with a beard and a beard sitting in fron...,a154a56b1b87cf3a0bc93600c86e4585,/r/KoreanHotties/comments/11hfioi/underboob_bi...,https://i.redd.it/cimr17q84jla1.jpg,a154a56b1b87cf3a0bc93600c86e4585.jpg,D:\data\images\KoreanHotties\a154a56b1b87cf3a0...,D:\data\images\KoreanHotties\thumbnail\a154a56...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23823,11oz7fi,mildlypenis,Extra-Lie489,Lenis,there is a small piece of metal with two small...,7604c3e55ce8dbf238520683d6e71f89,/r/mildlypenis/comments/11oz7fi/lenis/,https://i.redd.it/pskj3q0ao8na1.jpg,7604c3e55ce8dbf238520683d6e71f89.jpg,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,D:\data\images\mildlypenis\thumbnail\7604c3e55...,True,False
23824,11oz73d,mildlypenis,DeimosLV,My friend is making a stuffed animal and this ...,a close up of a blue knitted bagueth on a chair,389f8aa0fb191bdeba08c8d2cfff8ce2,/r/mildlypenis/comments/11oz73d/my_friend_is_m...,https://i.redd.it/gemqhq67o8na1.jpg,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,D:\data\images\mildlypenis\thumbnail\389f8aa0f...,True,False
23825,11oyzm3,mildlypenis,Mymemesareswell,This weird Mickey keychain hand,someone is holding a key chain with a mickey m...,87563ac0c8ddeeb0ce47bc915b99308b,/r/mildlypenis/comments/11oyzm3/this_weird_mic...,https://i.redd.it/vrivt8rom8na1.jpg,87563ac0c8ddeeb0ce47bc915b99308b.jpg,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,D:\data\images\mildlypenis\thumbnail\87563ac0c...,True,False
23826,11oyvx1,mildlypenis,LucindaBobinda,Parking lot pp,a close up of a black and white photo of a bla...,88b596be6de959bd29eed4e8946bb476,/r/mildlypenis/comments/11oyvx1/parking_lot_pp/,https://i.redd.it/w1vebizwl8na1.jpg,88b596be6de959bd29eed4e8946bb476.jpg,D:\data\images\mildlypenis\88b596be6de959bd29e...,D:\data\images\mildlypenis\thumbnail\88b596be6...,True,False


In [6]:
display(sources_df)

,name,data
0,CityDiffusion,[CityPorn]
1,NatureDiffusion,[EarthPorn]
2,CosmicDiffusion,[spaceporn]
3,ITAPDiffusion,[itookapicture]
4,MemeDiffusion,[memes]
5,TTTDiffusion,[trippinthroughtime]
6,WallStreetDiffusion,[wallstreetbets]
7,SexyDiffusion,"[selfies, Amicute, amihot, AmIhotAF, HotGirlNe..."
8,FatSquirrelDiffusion,[fatsquirrelhate]
9,CelebrityDiffusion,[celebrities]


In [7]:
smaller_exportable_df = pd.DataFrame(
	data=[exists['path'], exists['image_name'], exists['caption'], exists['title'], exists['subreddit']],
	index=['path', 'image_name', 'caption', 'title', 'subreddit']).T

display(smaller_exportable_df)

,path,image_name,caption,title,subreddit
0,D:\data\images\celebrities\f09b1c0c5dea50c9340...,f09b1c0c5dea50c934081a04c83d4d33.jpg,cars are parked on the side of the road in the...,Rachel Weisz,celebrities
1,D:\data\images\prettyasiangirls\9d5b7331b941ec...,9d5b7331b941ecf3fdbc8ad322303825.jpg,blonde woman with blonde hair and tattoos on h...,Library,prettyasiangirls
2,D:\data\images\KoreanHotties\a154a56b1b87cf3a0...,a154a56b1b87cf3a0bc93600c86e4585.jpg,a man with a beard and a beard sitting in fron...,Underboob bikini,KoreanHotties
3,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn
4,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn
...,...,...,...,...,...
23823,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,7604c3e55ce8dbf238520683d6e71f89.jpg,there is a small piece of metal with two small...,Lenis,mildlypenis
23824,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,a close up of a blue knitted bagueth on a chair,My friend is making a stuffed animal and this ...,mildlypenis
23825,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,87563ac0c8ddeeb0ce47bc915b99308b.jpg,someone is holding a key chain with a mickey m...,This weird Mickey keychain hand,mildlypenis
23826,D:\data\images\mildlypenis\88b596be6de959bd29e...,88b596be6de959bd29eed4e8946bb476.jpg,a close up of a black and white photo of a bla...,Parking lot pp,mildlypenis


In [8]:
smaller_exportable_df['name'] = smaller_exportable_df.progress_apply(lambda x: add_source(x, sources), axis=1)
display(smaller_exportable_df)

global: 100%|██████████| 23828/23828 [00:02<00:00, 9829.02it/s] 


,path,image_name,caption,title,subreddit,name
0,D:\data\images\celebrities\f09b1c0c5dea50c9340...,f09b1c0c5dea50c934081a04c83d4d33.jpg,cars are parked on the side of the road in the...,Rachel Weisz,celebrities,CelebrityDiffusion
1,D:\data\images\prettyasiangirls\9d5b7331b941ec...,9d5b7331b941ecf3fdbc8ad322303825.jpg,blonde woman with blonde hair and tattoos on h...,Library,prettyasiangirls,SexyAsianDiffusion
2,D:\data\images\KoreanHotties\a154a56b1b87cf3a0...,a154a56b1b87cf3a0bc93600c86e4585.jpg,a man with a beard and a beard sitting in fron...,Underboob bikini,KoreanHotties,SexyAsianDiffusion
3,/data/images/spaceporn/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,a view of a view of a large green and purple a...,Northern Lights above Lofoten,spaceporn,CosmicDiffusion
4,/data/images/spaceporn/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,a scene of a boat is sitting on the shore of a...,Viking Lights,spaceporn,CosmicDiffusion
...,...,...,...,...,...,...
23823,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,7604c3e55ce8dbf238520683d6e71f89.jpg,there is a small piece of metal with two small...,Lenis,mildlypenis,MildlyPenisDiffusion
23824,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,a close up of a blue knitted bagueth on a chair,My friend is making a stuffed animal and this ...,mildlypenis,MildlyPenisDiffusion
23825,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,87563ac0c8ddeeb0ce47bc915b99308b.jpg,someone is holding a key chain with a mickey m...,This weird Mickey keychain hand,mildlypenis,MildlyPenisDiffusion
23826,D:\data\images\mildlypenis\88b596be6de959bd29e...,88b596be6de959bd29eed4e8946bb476.jpg,a close up of a black and white photo of a bla...,Parking lot pp,mildlypenis,MildlyPenisDiffusion


In [ ]:
# !mkdir CelebrityDiffusion

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar():
	dask_frame = dd.from_pandas(smaller_exportable_df, npartitions=10)
	smaller_exportable_df['image_data'] = dask_frame.apply(lambda x: move_file(x), meta=('str', object),axis=1).compute()
display(smaller_exportable_df)

In [9]:
smaller_exportable_df.to_parquet("data.parquet")

In [43]:
extant_files = os.listdir("CelebrityDiffusion")
seen = []
things = []
records = smaller_exportable_df.to_dict('records')
for record in records:
	if record['image_name'] in extant_files:
		image_name = record['image_name']
		text = record['title']
		caption = record['caption']
		out_record = {"file_name": image_name, "text": [text, caption]}
		if image_name in seen:
			continue
		else:
			seen.append(image_name)
			things.append(out_record)
	else:
		print(f"File {record['image_name']} does not exist")

pd.DataFrame(things).to_json(f"CelebrityDiffusion/metadata.jsonl", orient="records", lines=True)

File 9d5b7331b941ecf3fdbc8ad322303825.jpg does not exist
File a154a56b1b87cf3a0bc93600c86e4585.jpg does not exist
File 7s5aafaqkb9a1.jpg does not exist
File abojw7lqlb9a1.jpg does not exist
File v3ra9g4vrb9a1.jpg does not exist
File ldz8tgpsub9a1.jpg does not exist
File qlr6w888ga9a1.jpg does not exist
File slyqoecbzb9a1.jpg does not exist
File 09kuna4hzb9a1.jpg does not exist
File el3s490lzb9a1.jpg does not exist
File u4n6sj6qac9a1.jpg does not exist
File 4y6hzocyec9a1.jpg does not exist
File 8xwnx4dlgc9a1.jpg does not exist
File 85y3hw6lnc9a1.jpg does not exist
File o4sh9b7csc9a1.jpg does not exist
File l19uhqt5jb9a1.jpg does not exist
File b6dqwg4t1d9a1.jpg does not exist
File geg8z6bv1d9a1.jpg does not exist
File a8kn2hka7d9a1.jpg does not exist
File 4w2einr1vb9a1.jpg does not exist
File hd5aixiwvb9a1.jpg does not exist
File 06iww0mced9a1.jpg does not exist
File ss9kdvlvhd9a1.jpg does not exist
File rdywcwiyhd9a1.jpg does not exist
File e2wdpy7mid9a1.jpg does not exist
File 737tgmr

In [13]:
import json

{"file_name":"zbd22gwlce9a1.jpg","text":["mmmm, banana bread...","a painting of a man in a red robe holding a knife and knife"]}
{"file_name":"hAGexJG.jpg","text":["First day on the job","there are two pictures of a woman in a dress and a man in a suit"]}
{"file_name":"xf7t1hffdi9a1.jpg","text":["Medieval manners","a picture of a picture of a woman holding a sword"]}
{"file_name":"CAVVBkn.jpg","text":["They ain't gonna happen","a picture taken from a photograph of a man and woman in a new year's resolution"]}
{"file_name":"cqu685xivk9a1.jpg","text":["we're is life were is death ?","a painting of a woman standing in front of a large painting"]}
{"file_name":"vZZrv7F.jpg","text":["Even the Cherub is disgusted","a picture taken from the bottom of a statue of a man in a suit"]}
{"file_name":"YAcg1Fy.jpg","text":["New shoes and a haircut","a picture taken from a picture of a man with a horse"]}
{"file_name":"o1un502dfr9a1.jpg","text":["Where art thou?","a picture taken from a picture of a m

In [67]:
extant_files = os.listdir("CelebrityDiffusion")

known_data_lines = []
for file in extant_files:
	if file.endswith(".jsonl"):
		continue
	else:
		if file in smaller_exportable_df['image_name'].values:
			filtered = smaller_exportable_df.where(smaller_exportable_df['image_name'] == file).dropna(how='all').reset_index(drop=True)
			image_name = filtered['image_name'][0]
			text = filtered['title'][0]
			caption = filtered['caption'][0]
			if image_name is None or text is None or caption is None:
				continue
			out_record = {"file_name": image_name, "text": [text, caption]}
			known_data_lines.append(out_record)
		else:
			continue

out = pd.DataFrame(data=known_data_lines)
display(out)

,file_name,text
0,0105751950dcfe5c88cf3af9de42be7f.jpg,"[Alexandra Daddario, a woman in a bikini top a..."
1,0216657afb8936b62aeb81036a6b841f.jpg,"[Alexandra Daddario, arafed image of a woman i..."
2,03e8db8c1ec0a201b680b23815aa07ba.jpg,"[Scarlett Johansson, a woman in a black dress ..."
3,0504d913c974a913eccd5884772da93c.jpg,"[Jenna Coleman, arafed image of a woman in a d..."
4,0516f02fa2a5020dc053aa3b938f165c.jpg,"[Allison Williams, a woman in a sequinous dres..."
...,...,...
302,fbfc5aff561655747534683c22532295.jpg,"[Dakota Johnson is love., a woman in a red dre..."
303,fc6626e3e27a020dc00b347832fa5b3f.jpg,"[Hailee Steinfeld, a woman in a white dress an..."
304,fe5009817fb04808f961d205ff95793d.jpg,"[Jennifer Lawrence., the blonde haired woman i..."
305,fee944b92931e26230a633f7c591f168.jpg,"[BLACKPINK, a group of girls in black dresses ..."


In [69]:
out.to_json(f"CelebrityDiffusion/metadata.jsonl", orient="records", lines=True)

In [34]:
# !powershell Compress-Archive CelebrityDiffusion CelebrityDiffusion.zip